First, we need to import several packages to use the APIs.

In [7]:
import pandas as pd

In [8]:
import requests
import os

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"Louisville, KY","status_type":"ForSale","sort":"status_type = ForSale"}

headers = {
	"X-RapidAPI-Key": os.environ.get('RAPIDAPI_KEY'),
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'Invalid API key. Go to https://docs.rapidapi.com/docs/keys for more info.'}


In [12]:
import os
try:
    env_var = os.environ['RAPIDAPI_KEY']
    print('ENV environment variable exists')
except KeyError:
    print('ENV environment variable does not exist')

ENV environment variable does not exist
